<a href="https://colab.research.google.com/github/tariqulq-cyber/Pemrosesan-teks-praktek/blob/main/scraping%20%26%20casefolding/gas1_pemrosesan_teks_teori_scraping_%26_casefolding_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
pip install google-play-scraper pandas

SCREPING

In [42]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import os
import textwrap # Import modul untuk memecah teks panjang agar tampilan rapi

# --- 1. KONFIGURASI DAN PARAMETER ---

APP_ID = 'com.rtsoft.growtopia'
LIMIT_KOMENTAR = 1000
TAHUN_BATAS_ATAS = 2025 # Batas tahun yang diminta: 2025 ke bawah
NAMA_FILE_OUTPUT = 'komentar_growtopia_2025_kebawah.csv'

# Batas tanggal eksklusif (1 Januari tahun berikutnya)
# Ulasan harus <= 31 Desember 2025
TANGGAL_BATAS = datetime(TAHUN_BATAS_ATAS + 1, 1, 1)

print("=======================================================")
print(f"TARGET: Scraping {LIMIT_KOMENTAR} komentar terbaru dari Growtopia")
print(f"FILTER: Komentar dengan tanggal <= 31 Desember {TAHUN_BATAS_ATAS}")
print("=======================================================")

# --- 2. PROSES SCRAPING (PENGAMBILAN DATA) ---

try:
    # Mengambil ulasan (review) terbaru
    # Ini adalah satu-satunya proses pengambilan data/scraping.
    result, continuation_token = reviews(
        APP_ID,
        lang='id',    # Bahasa Indonesia
        country='id', # Negara Indonesia
        sort=Sort.NEWEST, # Mengurutkan dari yang terbaru
        count=LIMIT_KOMENTAR,
        filter_score_with=None # Mengambil semua skor rating
    )

    # Mengubah hasil scraping menjadi DataFrame Pandas
    df_raw = pd.DataFrame(result)

    print(f"\n[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: {len(df_raw)}.")

except Exception as e:
    print(f"\n[ERROR] Terjadi kesalahan saat scraping: {e}")
    df_raw = pd.DataFrame()


# --- 3. PROSES FILTERING BERDASARKAN TANGGAL ---

df_scraped_filtered = pd.DataFrame()

if not df_raw.empty:

    # 3.1. Mempersiapkan kolom tanggal
    df_raw['at'] = pd.to_datetime(df_raw['at'])

    # 3.2. Menerapkan Filtering
    df_scraped_filtered = df_raw[df_raw['at'] < TANGGAL_BATAS].copy()

    # 3.3. Memilih dan menata kolom untuk output
    df_scraped_filtered = df_scraped_filtered[[
        'userName',
        'at',
        'content',
        'score',
        'thumbsUpCount'
    ]].reset_index(drop=True)

    # Menambahkan kolom NO dan mengubah format tanggal
    df_scraped_filtered.insert(0, 'NO', range(1, 1 + len(df_scraped_filtered)))
    df_scraped_filtered['at'] = df_scraped_filtered['at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    print(f"[INFO] Setelah filter tahun {TAHUN_BATAS_ATAS} ke bawah, sisa komentar: {len(df_scraped_filtered)}.")
    if len(df_scraped_filtered) == 0:
        print("[PERINGATAN] Tidak ada komentar dalam 1000 ulasan terbaru yang berasal dari tahun tersebut.")

else:
    print("\n[ERROR] Data scraping kosong, tidak dapat melakukan filtering.")


# --- 4. TAMPILAN OUTPUT LENGKAP (RATA KIRI DAN RAPI - TANPA CASEFOLDING) ---

if not df_scraped_filtered.empty:

    pd.reset_option('all')

    # --- Penyiapan Format Tabel Manual untuk Rata Kiri ---

    # Mendefinisikan lebar kolom:
    WIDTH_NO = 4
    WIDTH_USER = 20
    WIDTH_DATE = 20
    WIDTH_SCORE = 5
    WIDTH_THUMBS = 5

    # Menghitung lebar kolom konten untuk wrapping
    WIDTH_NON_CONTENT = WIDTH_NO + WIDTH_USER + WIDTH_DATE + WIDTH_SCORE + WIDTH_THUMBS + 10
    WIDTH_CONTENT_WRAP = 120 - WIDTH_NON_CONTENT - 4

    # Header Tabel (Diperbarui untuk mencerminkan bahwa konten adalah MENTAH)
    header = (
        f"{'NO':<{WIDTH_NO}} | {'USERNAME':<{WIDTH_USER}} | {'AT':<{WIDTH_DATE}} | "
        f"{'CONTENT (Full, RAW TEXT)':<{WIDTH_CONTENT_WRAP}} | {'SCORE':<{WIDTH_SCORE}} | {'SUKA':<{WIDTH_THUMBS}} |"
    )
    separator = "=" * len(header)

    # --- Mencetak Output ke Konsol ---

    print("\n\n=======================================================")
    print(f"HASIL AKHIR SCRAPING (Total Komentar Tampil: {len(df_scraped_filtered)})")
    print("=======================================================")
    print("CATATAN: Komentar LENGKAP ditampilkan sebagai Teks Mentah (RAW TEXT), sesuai aslinya.")
    print("-------------------------------------------------------------------")

    # Cetak Header dan Separator
    print(header)
    print(separator)

    # Cetak Isi Tabel (Looping dan Text Wrapping)
    for index, row in df_scraped_filtered.iterrows():
        # *** PERUBAHAN UTAMA: MENGGUNAKAN KONTEN MENTAH (TANPA .lower()) ***
        raw_content = str(row['content']) # Mengambil konten mentah

        # 2. Pecah konten panjang menjadi list baris
        wrapped_lines = textwrap.wrap(raw_content, width=WIDTH_CONTENT_WRAP)

        if not wrapped_lines:
            wrapped_lines = ["-"]

        # --- Cetak Baris Pertama (dengan semua data) ---
        line = (
            f"{row['NO']:<{WIDTH_NO}} | {row['userName']:<{WIDTH_USER}} | {row['at']:<{WIDTH_DATE}} | "
            f"{wrapped_lines[0]:<{WIDTH_CONTENT_WRAP}} | {row['score']:<{WIDTH_SCORE}} | {row['thumbsUpCount']:<{WIDTH_THUMBS}} |"
        )
        print(line)

        # --- Cetak Baris Lanjutan (dengan indentasi) ---
        for subsequent_line in wrapped_lines[1:]:

            # Kolom Non-Content diganti dengan spasi kosong
            blank_columns = (
                f"{'':<{WIDTH_NO}} | {'':<{WIDTH_USER}} | {'':<{WIDTH_DATE}} | "
            )

            # Baris lanjutan hanya berisi indentasi, konten, dan spasi kosong di kolom akhir
            continuation_line = (
                f"{blank_columns}{subsequent_line:<{WIDTH_CONTENT_WRAP}} | {'':<{WIDTH_SCORE}} | {'':<{WIDTH_THUMBS}} |"
            )
            print(continuation_line)

    print(separator)




TARGET: Scraping 1000 komentar terbaru dari Growtopia
FILTER: Komentar dengan tanggal <= 31 Desember 2025

[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: 1000.
[INFO] Setelah filter tahun 2025 ke bawah, sisa komentar: 1000.


HASIL AKHIR SCRAPING (Total Komentar Tampil: 1000)
CATATAN: Komentar LENGKAP ditampilkan sebagai Teks Mentah (RAW TEXT), sesuai aslinya.
-------------------------------------------------------------------
NO   | USERNAME             | AT                   | CONTENT (Full, RAW TEXT)                             | SCORE | SUKA  |
1    | YT•RIZ GANT OFFICIAL | 2025-10-21 15:36:44  | game nya susah banget anj cuman buat login bikin     | 1     | 0     |
     |                      |                      | akun di persulit game konfl                          |       |       |
2    | Bang Rama            | 2025-10-21 14:57:10  | 🤑                                                    | 5     | 0     |
3    | muhammad ridho official | 2025-10-21 14:42:42  |

/tmp/ipython-input-1411266247.py:84: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/tmp/ipython-input-1411266247.py:84: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [43]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import os
import textwrap
import sys # Ditambahkan untuk logging error
import atexit # Ditambahkan untuk memastikan file dibuka/ditutup dengan benar

# --- 1. KONFIGURASI DAN PARAMETER ---

APP_ID = 'com.rtsoft.growtopia'
LIMIT_KOMENTAR = 1000
TAHUN_BATAS_ATAS = 2025
NAMA_FILE_OUTPUT = 'screping_growtopia.csv'
TANGGAL_BATAS = datetime(TAHUN_BATAS_ATAS + 1, 1, 1)

print("=======================================================")
print(f"TARGET: Scraping {LIMIT_KOMENTAR} komentar terbaru dari Growtopia")
print(f"FILTER: Komentar dengan tanggal <= 31 Desember {TAHUN_BATAS_ATAS}")
print("=======================================================")

# --- 2. PROSES SCRAPING (PENGAMBILAN DATA) ---

try:
    result, continuation_token = reviews(
        APP_ID,
        lang='id',
        country='id',
        sort=Sort.NEWEST,
        count=LIMIT_KOMENTAR,
        filter_score_with=None
    )

    df_raw = pd.DataFrame(result)

    print(f"\n[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: {len(df_raw)}.")

except Exception as e:
    print(f"\n[ERROR] Terjadi kesalahan saat scraping: {e}")
    df_raw = pd.DataFrame()


# --- 3. PROSES FILTERING BERDASARKAN TANGGAL DAN PENATAAN KOLOM ---

df_scraped_filtered = pd.DataFrame()
df_export = pd.DataFrame() # Inisialisasi

if not df_raw.empty:

    df_raw['at'] = pd.to_datetime(df_raw['at'])
    df_scraped_filtered = df_raw[df_raw['at'] < TANGGAL_BATAS].copy()

    # Pilih dan tata kolom
    df_scraped_filtered = df_scraped_filtered[[
        'userName', 'at', 'content', 'score', 'thumbsUpCount'
    ]].reset_index(drop=True)

    # Menambahkan kolom NO dan mengubah format tanggal untuk tampilan konsol
    df_scraped_filtered.insert(0, 'NO', range(1, 1 + len(df_scraped_filtered)))
    df_scraped_filtered['at'] = df_scraped_filtered['at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Membuat salinan DataFrame untuk ekspor (TANPA kolom 'NO')
    df_export = df_scraped_filtered.drop(columns=['NO']).copy()

    print(f"[INFO] Setelah filter tahun {TAHUN_BATAS_ATAS} ke bawah, sisa komentar: {len(df_export)}.")
    if len(df_export) == 0:
        print("[PERINGATAN] Tidak ada komentar dalam 1000 ulasan terbaru yang berasal dari tahun tersebut.")
        sys.exit(0) # Keluar jika tidak ada data untuk diekspor

else:
    print("\n[ERROR] Data scraping kosong, tidak dapat melakukan filtering atau ekspor.")
    sys.exit(1) # Keluar jika scraping gagal


# --- 4. TAMPILAN OUTPUT LENGKAP (ke konsol) ---
# ... (Kode untuk mencetak tabel ke konsol seperti yang sudah ada, dilewati untuk ringkasan) ...

if not df_scraped_filtered.empty:
    pd.reset_option('all')

    # ... (Kode cetak yang panjang, tidak ditampilkan di sini, asumsikan sudah benar) ...
    # print(separator)

    # ... (lanjutkan ke Langkah 5) ...


## 5. EKSPOR DATA KE FILE CSV (FOKUS SOLUSI) 💾

if not df_export.empty:
    try:
        # PENTING: Gunakan 'w' (write) untuk mode penulisan dan pastikan file ditutup
        with open(NAMA_FILE_OUTPUT, 'w', newline='', encoding='utf-8') as f:
            # Menggunakan .to_csv pada file handler
            df_export.to_csv(
                f, # Menulis ke file handler 'f'
                index=False,
                encoding='utf-8',
                errors='ignore' # Tambahkan ini untuk mengabaikan karakter yang bermasalah
            )

        print(f"\n[SUKSES] Data berhasil diekspor ke file: {os.path.abspath(NAMA_FILE_OUTPUT)}")
        print("=======================================================")

    except Exception as e:
        # LOGGING ERROR KRITIS
        print("=======================================================")
        print("\n[!!! KEGAGALAN KRITIS EKSPOR CSV !!!]")
        print(f"Tipe Kesalahan: {type(e).__name__}")
        print(f"Detail Kesalahan: {e}")
        print(f"Jalur file yang dicoba: {os.path.abspath(NAMA_FILE_OUTPUT)}")
        print("\nSOLUSI:")
        print("1. Pastikan file CSV tidak sedang dibuka di program lain (Excel, editor teks).")
        print("2. Coba ganti NAMA_FILE_OUTPUT dengan jalur ABSOLUT (lihat komentar di atas).")
        print("3. Coba jalankan terminal/CMD Anda sebagai ADMINISTRATOR.")
        print("=======================================================")

TARGET: Scraping 1000 komentar terbaru dari Growtopia
FILTER: Komentar dengan tanggal <= 31 Desember 2025

[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: 1000.
[INFO] Setelah filter tahun 2025 ke bawah, sisa komentar: 1000.

[SUKSES] Data berhasil diekspor ke file: /content/screping_growtopia.csv


/tmp/ipython-input-187788541.py:79: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/tmp/ipython-input-187788541.py:79: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


CASEFOLDING


In [44]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import os
import textwrap # Import modul untuk memecah teks panjang

# --- 1. KONFIGURASI DAN PARAMETER ---

APP_ID = 'com.rtsoft.growtopia'
LIMIT_KOMENTAR = 1000
TAHUN_BATAS_ATAS = 2025 # Batas tahun yang diminta: 2025 ke bawah
NAMA_FILE_OUTPUT = 'komentar_growtopia_2025_kebawah.csv'

# Batas tanggal eksklusif (1 Januari tahun berikutnya)
# Ulasan harus <= 31 Desember 2025
TANGGAL_BATAS = datetime(TAHUN_BATAS_ATAS + 1, 1, 1)

print("=======================================================")
print(f"TARGET: Scraping {LIMIT_KOMENTAR} komentar terbaru dari Growtopia")
print(f"FILTER: Komentar dengan tanggal <= 31 Desember {TAHUN_BATAS_ATAS}")
print("=======================================================")

# --- 2. PROSES SCRAPING (PENGAMBILAN DATA) ---

try:
    # Mengambil ulasan (review) terbaru
    result, continuation_token = reviews(
        APP_ID,
        lang='id',    # Bahasa Indonesia
        country='id', # Negara Indonesia
        sort=Sort.NEWEST, # Mengurutkan dari yang terbaru
        count=LIMIT_KOMENTAR,
        filter_score_with=None # Mengambil semua skor rating
    )

    # Mengubah hasil scraping menjadi DataFrame Pandas
    df_raw = pd.DataFrame(result)

    print(f"\n[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: {len(df_raw)}.")

except Exception as e:
    print(f"\n[ERROR] Terjadi kesalahan saat scraping: {e}")
    df_raw = pd.DataFrame()


# --- 3. PROSES FILTERING BERDASARKAN TANGGAL ---

df_scraped_filtered = pd.DataFrame()

if not df_raw.empty:

    # 3.1. Mempersiapkan kolom tanggal
    df_raw['at'] = pd.to_datetime(df_raw['at'])

    # 3.2. Menerapkan Filtering
    df_scraped_filtered = df_raw[df_raw['at'] < TANGGAL_BATAS].copy()

    # 3.3. Memilih dan menata kolom untuk output
    df_scraped_filtered = df_scraped_filtered[[
        'userName',
        'at',
        'content',
        'score',
        'thumbsUpCount'
    ]].reset_index(drop=True)

    # Menambahkan kolom NO dan mengubah format tanggal
    df_scraped_filtered.insert(0, 'NO', range(1, 1 + len(df_scraped_filtered)))
    df_scraped_filtered['at'] = df_scraped_filtered['at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    print(f"[INFO] Setelah filter tahun {TAHUN_BATAS_ATAS} ke bawah, sisa komentar: {len(df_scraped_filtered)}.")
    if len(df_scraped_filtered) == 0:
        print("[PERINGATAN] Tidak ada komentar dalam 1000 ulasan terbaru yang berasal dari tahun tersebut.")

else:
    print("\n[ERROR] Data scraping kosong, tidak dapat melakukan filtering.")


# --- 4. TAMPILAN OUTPUT LENGKAP (RATA KIRI DAN RAPI DENGAN TEXT WRAPPING) ---

if not df_scraped_filtered.empty:

    # Menghapus pengaturan Pandas yang tidak efektif untuk perataan konsol
    pd.reset_option('all')

    # --- Penyiapan Format Tabel Manual untuk Rata Kiri ---

    # Mendefinisikan lebar kolom:
    WIDTH_NO = 4
    WIDTH_USER = 20
    WIDTH_DATE = 20
    WIDTH_SCORE = 5
    WIDTH_THUMBS = 5

    # Lebar total untuk kolom selain content:
    # (3 spasi | 20 spasi | 3 spasi | 20 spasi | 3 spasi) = 49 spasi
    WIDTH_NON_CONTENT = WIDTH_NO + WIDTH_USER + WIDTH_DATE + WIDTH_SCORE + WIDTH_THUMBS + 10 # Total spasi dan pemisah: ~64

    # Lebar kolom konten untuk wrapping
    # Sisa lebar layar (misalnya 120) dikurangi lebar non-konten.
    WIDTH_CONTENT_WRAP = 120 - WIDTH_NON_CONTENT - 4 # Sekitar 50-55 karakter per baris wrap

    # Header Tabel
    header = (
        f"{'NO':<{WIDTH_NO}} | {'USERNAME':<{WIDTH_USER}} | {'AT':<{WIDTH_DATE}} | "
        f"{'CONTENT (Casefolded, Full)':<{WIDTH_CONTENT_WRAP}} | {'SCORE':<{WIDTH_SCORE}} | {'SUKA':<{WIDTH_THUMBS}} |"
    )
    separator = "=" * len(header)

    # String kosong yang akan digunakan sebagai indentasi untuk baris lanjutan konten
    INDENT_SPACER = " " * (WIDTH_NO + 1 + 1 + WIDTH_USER + 1 + 1 + WIDTH_DATE + 1 + 1) # Total 50 spasi sebelum konten

    # --- Mencetak Output ke Konsol ---

    print("\n\n=======================================================")
    print(f"HASIL AKHIR SCRAPING (Total Komentar Tampil: {len(df_scraped_filtered)})")
    print("=======================================================")
    print("CATATAN: Komentar LENGKAP ditampilkan dengan Text Wrapping Rata Kiri.")
    print("=======================================================")
    print("CATATAN: Teks komentar ('content') adalah data yang sudah melakukan casefolding.")
    print("---------------------------------------------------------------------------------------------------------------------------")

    # Cetak Header dan Separator
    print(header)
    print(separator)

    # Cetak Isi Tabel (Looping dan Text Wrapping)
    for index, row in df_scraped_filtered.iterrows():
        # 1. Terapkan Casefolding
        content_casefolded = str(row['content']).lower()

        # 2. Pecah konten panjang menjadi list baris
        wrapped_lines = textwrap.wrap(content_casefolded, width=WIDTH_CONTENT_WRAP)

        if not wrapped_lines:
            # Handle jika konten kosong
            wrapped_lines = ["-"]

        # --- Cetak Baris Pertama (dengan semua data) ---
        line = (
            f"{row['NO']:<{WIDTH_NO}} | {row['userName']:<{WIDTH_USER}} | {row['at']:<{WIDTH_DATE}} | "
            f"{wrapped_lines[0]:<{WIDTH_CONTENT_WRAP}} | {row['score']:<{WIDTH_SCORE}} | {row['thumbsUpCount']:<{WIDTH_THUMBS}} |"
        )
        print(line)

        # --- Cetak Baris Lanjutan (dengan indentasi) ---
        for subsequent_line in wrapped_lines[1:]:
            # Format baris lanjutan: menggunakan spasi kosong untuk kolom non-konten
            # dan memastikan konten lanjutan tetap Rata Kiri

            # Kolom Non-Content diganti dengan spasi kosong
            blank_columns = (
                f"{'':<{WIDTH_NO}} | {'':<{WIDTH_USER}} | {'':<{WIDTH_DATE}} | "
            )

            # Baris lanjutan hanya berisi indentasi, konten, dan spasi kosong di kolom akhir
            continuation_line = (
                f"{blank_columns}{subsequent_line:<{WIDTH_CONTENT_WRAP}} | {'':<{WIDTH_SCORE}} | {'':<{WIDTH_THUMBS}} |"
            )
            print(continuation_line)

    print(separator)


TARGET: Scraping 1000 komentar terbaru dari Growtopia
FILTER: Komentar dengan tanggal <= 31 Desember 2025

[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: 1000.
[INFO] Setelah filter tahun 2025 ke bawah, sisa komentar: 1000.


HASIL AKHIR SCRAPING (Total Komentar Tampil: 1000)
CATATAN: Komentar LENGKAP ditampilkan dengan Text Wrapping Rata Kiri.
CATATAN: Teks komentar ('content') adalah data yang sudah melakukan casefolding.
---------------------------------------------------------------------------------------------------------------------------
NO   | USERNAME             | AT                   | CONTENT (Casefolded, Full)                           | SCORE | SUKA  |
1    | YT•RIZ GANT OFFICIAL | 2025-10-21 15:36:44  | game nya susah banget anj cuman buat login bikin     | 1     | 0     |
     |                      |                      | akun di persulit game konfl                          |       |       |
2    | Bang Rama            | 2025-10-21 14:57:10  | 🤑    

/tmp/ipython-input-2889980804.py:84: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/tmp/ipython-input-2889980804.py:84: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [45]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import os
import textwrap
import sys

# --- 1. KONFIGURASI DAN PARAMETER ---

APP_ID = 'com.rtsoft.growtopia'
LIMIT_KOMENTAR = 1000
TAHUN_BATAS_ATAS = 2025
NAMA_FILE_OUTPUT = 'casefolding_growtopia.csv'

TANGGAL_BATAS = datetime(TAHUN_BATAS_ATAS + 1, 1, 1)

print("=======================================================")
print(f"TARGET: Scraping {LIMIT_KOMENTAR} komentar terbaru dari Growtopia")
print(f"FILTER: Komentar dengan tanggal <= 31 Desember {TAHUN_BATAS_ATAS}")
print("=======================================================")

# --- 2. PROSES SCRAPING (PENGAMBILAN DATA) ---

try:
    result, continuation_token = reviews(
        APP_ID,
        lang='id',
        country='id',
        sort=Sort.NEWEST,
        count=LIMIT_KOMENTAR,
        filter_score_with=None
    )

    df_raw = pd.DataFrame(result)

    print(f"\n[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: {len(df_raw)}.")

except Exception as e:
    print(f"\n[ERROR] Terjadi kesalahan saat scraping: {e}")
    df_raw = pd.DataFrame()


# --- 3. PROSES FILTERING, CLEANING, DAN CASEFOLDING ---

df_scraped_filtered = pd.DataFrame()
df_export = pd.DataFrame()

if not df_raw.empty:

    # 3.1. Mempersiapkan kolom tanggal dan filtering
    df_raw['at'] = pd.to_datetime(df_raw['at'])
    df_scraped_filtered = df_raw[df_raw['at'] < TANGGAL_BATAS].copy()

    if df_scraped_filtered.empty:
        print("[PERINGATAN] Tidak ada komentar dalam 1000 ulasan terbaru yang berasal dari tahun tersebut.")
        sys.exit(0)

    # 3.2. Memilih kolom
    df_scraped_filtered = df_scraped_filtered[[
        'userName', 'at', 'content', 'score', 'thumbsUpCount'
    ]].reset_index(drop=True)

    # 3.3. Mengubah format tanggal
    df_scraped_filtered['at'] = df_scraped_filtered['at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # 3.4. *** CASEFOLDING ***
    # Mengubah semua isi kolom 'content' menjadi huruf kecil (lowercase)
    df_scraped_filtered['content_casefolded'] = df_scraped_filtered['content'].astype(str).str.lower()

    # 3.5. Membuat DataFrame untuk ekspor (hanya kolom yang diperlukan)
    df_export = df_scraped_filtered[['userName', 'at', 'content_casefolded', 'score', 'thumbsUpCount']].copy()
    df_export.columns = ['userName', 'at', 'content', 'score', 'thumbsUpCount'] # Ubah nama kolom kembali ke 'content'

    # Menambahkan kolom NO untuk tampilan konsol
    df_scraped_filtered.insert(0, 'NO', range(1, 1 + len(df_scraped_filtered)))

    print(f"[INFO] Setelah filter dan casefolding, data siap diekspor: {len(df_export)} baris.")

else:
    print("\n[ERROR] Data scraping kosong, tidak dapat melakukan ekspor.")
    sys.exit(1)


# --- 4. TAMPILAN OUTPUT LENGKAP (opsional, untuk melihat hasil casefolding) ---

if not df_scraped_filtered.empty:

    pd.reset_option('all')

    # ... (Kode cetak untuk konsol yang menampilkan kolom 'content_casefolded')
    # ** CATATAN **: Untuk kesederhanaan, kode untuk mencetak tampilan tabel lengkap di konsol DILOMPATI.
    # Namun, di script Anda, pastikan Anda mencetak kolom 'content_casefolded' agar terlihat perubahannya.
    # ... (lanjutkan ke Langkah 5) ...

    # =======================================================
    # Hanya untuk menunjukkan hasil casefolding di konsol (Contoh 3 baris teratas)
    print("\n\n=======================================================")
    print("HASIL CASEFOLDING (3 Baris Contoh)")
    print("=======================================================")
    print(df_scraped_filtered[['NO', 'content_casefolded']].head(3).to_string(index=False))
    print("=======================================================")


# --- 5. EKSPOR DATA KE FILE CSV (FOKUS SOLUSI) 💾 ---

if not df_export.empty:
    try:
        # Menggunakan 'with open' untuk keamanan penulisan file
        with open(NAMA_FILE_OUTPUT, 'w', newline='', encoding='utf-8') as f:
            df_export.to_csv(
                f, # Menulis ke file handler 'f'
                index=False,
                encoding='utf-8',
                errors='ignore'
            )

        print(f"\n[SUKSES EKSPOR] Data CASEFOLDED berhasil disimpan ke file: {os.path.abspath(NAMA_FILE_OUTPUT)}")
        print("=======================================================")

    except Exception as e:
        print("=======================================================")
        print("\n[!!! KEGAGALAN KRITIS EKSPOR CSV !!!]")
        print(f"Tipe Kesalahan: {type(e).__name__}")
        print(f"Detail Kesalahan: {e}")
        print(f"Jalur file yang dicoba: {os.path.abspath(NAMA_FILE_OUTPUT)}")
        print("\n*** Solusi Utama: Pastikan file CSV tidak sedang dibuka & coba jalankan sebagai Administrator. ***")
        print("=======================================================")

TARGET: Scraping 1000 komentar terbaru dari Growtopia
FILTER: Komentar dengan tanggal <= 31 Desember 2025

[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: 1000.
[INFO] Setelah filter dan casefolding, data siap diekspor: 1000 baris.


HASIL CASEFOLDING (3 Baris Contoh)
 NO                                                           content_casefolded
  1 game nya susah banget anj cuman buat login bikin akun di persulit game konfl
  2                                                                            🤑
  3                                                                     not good

[SUKSES EKSPOR] Data CASEFOLDED berhasil disimpan ke file: /content/casefolding_growtopia.csv


/tmp/ipython-input-2675707297.py:88: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/tmp/ipython-input-2675707297.py:88: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')
